In [3]:
import pandas as pd
import numpy as np
import re

# 1. 数据预处理

In [45]:
data = pd.read_csv('sohu.csv')
data[0:10]

,label,text
0,pic,高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑...
1,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
2,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
3,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
4,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
5,pic,10月4日十佳图：李娜小德球迷抢眼 大威显文静责编:马克杰 日期:2013-10-4 小德的...
6,pic,高清：小德速胜高举双臂庆祝 鼓掌向球迷们致意责编:马克杰 日期:2013-10-4 小德鼓掌...
7,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
8,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
9,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."


In [46]:
data.loc[0][1]

'高清：彭帅谢淑薇遗憾遭逆转出局 击掌互相鼓励责编:马克杰 日期:2013-10-4 彭帅谢淑薇互相鼓励 北京时间10月4日消息,2013年中国网球公开赛结束了一场女双半决赛的争夺,赛会2号种子――我国金花彭帅和中华台北名将谢淑薇的海峡组合错过了先胜一盘、抢十局8-4领先的优势,连丢六分之后以6-3/1-6/8-10惨遭逆转,无缘女双决赛。(搜狐-李志岩/摄)'

## 统计每个label下的新闻

In [47]:
#每个label的新闻条数
data['label'].value_counts()

news        2989
sports      1200
business    1051
pic          312
yule         185
mil           95
caipiao       45
cul           44
Name: label, dtype: int64

#### 由于最后5类的数据较少，只选择news, sports, business三类的新闻进行训练

In [48]:
#每条新闻的长度
data = data[~data['label'].str.contains('cul')] 
data = data[~data['label'].str.contains('caipiao')] 
data = data[~data['label'].str.contains('mil')] 
data = data[~data['label'].str.contains('yule')] 
data = data[~data['label'].str.contains('pic')] 
data['label'].value_counts()

news        2989
sports      1200
business    1051
Name: label, dtype: int64

In [49]:
#注意去掉一些列之后要重新设索引
data = data.reset_index(drop=True)

In [50]:
data[0:10]

,label,text
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走..."
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争..."
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...
5,news,美“宙斯盾”系统成功进行反导试验新华网华盛顿10月4日电(记者林小春)美国国防部4日宣布再次...
6,news,"3中国建筑工俄罗斯身亡【3中国建筑工俄罗斯身亡】俄伊尔库茨克州称,4日,1名中国建筑工人从在..."
7,news,山西煤矿透水事故排水5万立方米 仍有12人被困中新社汾阳10月4日电(张云)山西汾西矿业正升...
8,sports,"现场：彭帅暴力发球遭无视 主裁秀中文“谢谢”北京时间9月30日消息,总奖金额为5,185,6..."
9,news,"扬州警方：一名游客在扬州博物馆自主坠楼身亡【一游客在扬州坠亡】扬州警方通报,今天下午4点多,..."


In [51]:
data?

## 统计每条新闻的长度

In [52]:
data['length'] = 0
#data数组共5240行
data.head()

,label,text,length
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",0
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,0
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",0
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,0
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,0


In [54]:
for i in range(0, 5240):
    data['length'][i] = len(data['text'][i])

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### 删除长度小于50的新闻

In [57]:
data = data.drop(data[data['length']<50].index)

In [58]:
data.head()

,label,text,length
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",542
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,289
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",253
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,162
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,713


## 将标签和内容映射为数字

### 标签

In [59]:
label_dict={'news':1, 'sports':2, 'business':3}
data['l2id'] = data['label'].apply(lambda x: label_dict[x])

In [60]:
data.head()

,label,text,length,l2id
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",542,1
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,289,1
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",253,1
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,162,1
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,713,2


### 内容

In [61]:
data['chars']=data['text'].apply(lambda x: re.findall('[\x80-\xff]{3}|[\w\W]',x))

In [62]:
data.head()

,label,text,length,l2id,chars
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",542,1,"[河, 南, 周, 口, 路, 边, 秸, 秆, 燃, 烧, , 2, 岁, 幼, 儿, ..."
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,289,1,"[消, 息, 称, 1, 3, 8, 名, 中, 国, 劳, 工, 在, 菲, 律, 宾, ..."
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",253,1,"[越, 南, 重, 要, 军, 事, 领, 导, 人, 武, 元, 甲, 大, 将, 去, ..."
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,162,1,"[西, 沙, 搜, 救, 已, 发, 现, 6, 具, 遇, 难, 渔, 民, 遗, 体, ..."
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,713,2,"[卡, 努, 特, 伤, 愈, 将, 首, 发, 战, 辽, 足, , 斯, 塔, 诺, ..."


In [63]:
all_chars = []
for c in data['chars']:
    all_chars.extend(c)
char_dict = pd.DataFrame(pd.Series(all_chars).value_counts())
char_dict['id'] = list(range(1, len(char_dict)+1))

In [64]:
data['c2v'] = data['chars'].apply(lambda x: list(char_dict['id'][x]))

In [65]:
data.head()

,label,text,length,l2id,chars,c2v
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",542,1,"[河, 南, 周, 口, 路, 边, 秸, 秆, 燃, 烧, , 2, 岁, 幼, 儿, ...","[711, 185, 288, 318, 174, 461, 3420, 3409, 895..."
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,289,1,"[消, 息, 称, 1, 3, 8, 名, 中, 国, 劳, 工, 在, 菲, 律, 宾, ...","[243, 308, 268, 5, 28, 78, 120, 12, 14, 1057, ..."
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",253,1,"[越, 南, 重, 要, 军, 事, 领, 导, 人, 武, 元, 甲, 大, 将, 去, ...","[528, 185, 141, 53, 267, 113, 299, 294, 10, 76..."
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,162,1,"[西, 沙, 搜, 救, 已, 发, 现, 6, 具, 遇, 难, 渔, 民, 遗, 体, ...","[144, 925, 841, 339, 143, 31, 57, 47, 606, 641..."
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,713,2,"[卡, 努, 特, 伤, 愈, 将, 首, 发, 战, 辽, 足, , 斯, 塔, 诺, ...","[422, 1077, 180, 332, 1814, 70, 241, 31, 220, ..."


## 截取每条新闻前100个字

In [70]:
from keras.preprocessing import sequence
maxlen = 100
data['c2v'] = list(sequence.pad_sequences(data['c2v'], maxlen=maxlen))

In [71]:
data.head()

,label,text,length,l2id,chars,c2v
0,news,"河南周口路边秸秆燃烧 2岁幼儿被熏晕(图) 10月3日下午,崔先生驾车去周口郸城县汲冢镇走...",542,1,"[河, 南, 周, 口, 路, 边, 秸, 秆, 燃, 烧, , 2, 岁, 幼, 儿, ...","[256, 784, 339, 1, 186, 49, 1229, 710, 143, 72..."
1,news,消息称138名中国劳工在菲律宾被抓 使馆正核实【使馆正核实情况】中国驻菲律宾大使馆发言人表示...,289,1,"[消, 息, 称, 1, 3, 8, 名, 中, 国, 劳, 工, 在, 菲, 律, 宾, ...","[185, 2, 859, 190, 2447, 381, 2, 6, 32, 228, 1..."
2,news,"越南重要军事领导人武元甲大将去世 享年102岁据美国媒体10月4日报道, 越南抗法、抗美战争...",253,1,"[越, 南, 重, 要, 军, 事, 领, 导, 人, 武, 元, 甲, 大, 将, 去, ...","[1151, 794, 209, 299, 294, 2, 528, 185, 997, 3..."
3,news,西沙搜救已发现6具遇难渔民遗体 尚有52人失踪【已发现6具遇难渔民遗体 尚有52人失踪】10...,162,1,"[西, 沙, 搜, 救, 已, 发, 现, 6, 具, 遇, 难, 渔, 民, 遗, 体, ...","[841, 339, 10, 85, 562, 31, 57, 15, 47, 606, 6..."
4,sports,卡努特伤愈将首发战辽足 斯塔诺：辽足不好对付高清:国安踩场卡努特卖力 对抗赛神勇奔跑如风 (...,713,2,"[卡, 努, 特, 伤, 愈, 将, 首, 发, 战, 辽, 足, , 斯, 塔, 诺, ...","[18, 48, 75, 30, 288, 6, 357, 460, 476, 128, 1..."


## 生成训练集和测试集

In [103]:
from sklearn.model_selection import train_test_split

#测试集占20%
X = np.array(list(data['c2v']))
Y = np.array(list(data['l2id']))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=7)

这里要对Y进行to_categorical的处理，将其变成one-hot形式
因为分类器往往默认数据是连续有序的，但是上述数字是随机分配的
使用one-hot使得这些特征互斥，每次只有一个激活

In [105]:
y_train[0:3]

array([2, 2, 2])

In [106]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [107]:
y_train.shape

(4187, 4)

In [108]:
y_test.shape

(1047, 4)

# 建立模型

In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import LSTM

In [111]:
model = Sequential()
model.add(Embedding(len(char_dict)+1, 256)) 
model.add(LSTM(256))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 训练

In [112]:
model.fit(x_train, y_train, batch_size=128, epochs=20)

Train on 4187 samples
Epoch 1/20
4187/4187 [==============================] - 15s 4ms/sample - loss: 0.9942 - accuracy: 0.6274
Epoch 2/20
4187/4187 [==============================] - 16s 4ms/sample - loss: 0.5095 - accuracy: 0.7968
Epoch 3/20
4187/4187 [==============================] - 17s 4ms/sample - loss: 0.3405 - accuracy: 0.8701
Epoch 4/20
4187/4187 [==============================] - 18s 4ms/sample - loss: 0.2705 - accuracy: 0.8920
Epoch 5/20
4187/4187 [==============================] - 17s 4ms/sample - loss: 0.2071 - accuracy: 0.9183
Epoch 6/20
4187/4187 [==============================] - 16s 4ms/sample - loss: 0.1576 - accuracy: 0.9398
Epoch 7/20
4187/4187 [==============================] - 16s 4ms/sample - loss: 0.1401 - accuracy: 0.9491
Epoch 8/20
4187/4187 [==============================] - 16s 4ms/sample - loss: 0.1227 - accuracy: 0.9561
Epoch 9/20
4187/4187 [==============================] - 16s 4ms/sample - loss: 0.1064 - accuracy: 0.9601
Epoch 10/20
4187/4187 [==========

In [113]:
model.evaluate(x=x_test, y=y_test)

1047/1047 [==============================] - 3s 3ms/sample - loss: 0.8073 - accuracy: 0.8357


[0.8073433626802512, 0.83572114]

可以看到，虽然在train set中loss下降，accuracy增加

但在test set中，loss较大

由于数据集太小，出现了过拟合的问题